A simple implementation of Bidirectional RNN for sentiment analysis

Code from Geekforgeeks: https://www.geeksforgeeks.org/deep-learning/bidirectional-recurrent-neural-network/

Python packages to import
tensorflow

In [1]:
import warnings
warnings.filterwarnings('ignore')
# We will use the IMDB dataset built in Keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences

features = 2000  # Number of most frequent words to consider/time window
max_len = 50     # Maximum length of each sequence

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=features)

# Data Preprocessing: Padding sequences to ensure uniform input size
# Separating the words from the document into sequences of fixed length
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [2]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, SimpleRNN, Dense

embedding_dim = 128  
hidden_units = 64    

model = Sequential()
# Embedding() layer maps input features to dense vectors of size embedding (128), with an input length of len.
model.add(Embedding(features, embedding_dim, input_length=max_len))

# Bidirectional(SimpleRNN(hidden)) adds a bidirectional RNN layer with hidden (64) units.
model.add(Bidirectional(SimpleRNN(hidden_units)))

# Dense(1, activation='sigmoid') adds a dense output layer with 1 unit and a sigmoid activation for binary classification.
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [3]:
# Train the model
batch_size = 32
epochs = 5

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.7242 - loss: 0.5376 - val_accuracy: 0.7779 - val_loss: 0.4858
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8066 - loss: 0.4218 - val_accuracy: 0.7944 - val_loss: 0.4581
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.8389 - loss: 0.3648 - val_accuracy: 0.7468 - val_loss: 0.5277
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.8823 - loss: 0.2852 - val_accuracy: 0.7620 - val_loss: 0.5574
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9242 - loss: 0.1968 - val_accuracy: 0.7559 - val_loss: 0.6619


In [4]:
loss, accuracy = model.evaluate(X_test, y_test)

print('Test accuracy:', accuracy)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7559 - loss: 0.6619
Test accuracy: 0.7558799982070923


In [5]:
# Use test data for predictions
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

y_pred = (y_pred > 0.5)

print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
              precision    recall  f1-score   support

    Negative       0.75      0.76      0.76     12500
    Positive       0.76      0.75      0.75     12500

    accuracy                           0.76     25000
   macro avg       0.76      0.76      0.76     25000
weighted avg       0.76      0.76      0.76     25000



In [ ]:
# Test the model on sample strings
from keras.datasets import imdb

# Get the word index from IMDB dataset
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Decode function to convert encoded text back to words
# because the IMDB dataset encodes words as integers
def decode_review(encoded_text):
    """Convert encoded text back to words"""
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_text])

# Encode function to convert text to indices
# Which the BRNN model can process
def encode_text(text):
    """Convert text string to encoded indices"""
    words = text.lower().split()
    encoded = []
    for word in words:
        if word in word_index and word_index[word] < features:
            encoded.append(word_index[word])
    return encoded

# Test samples
# You can modify these strings to test different inputs
test_samples = [
    "The movie was fantastic! I really loved it.",
    "This film is terrible and boring. Waste of time.",
    "Amazing performance by the actors. Highly recommended!",
    "Worst movie I've ever seen. Absolutely horrible."
]

print("Testing BRNN Model on Sample Strings:\n")
print("-" * 60)

for test_text in test_samples:
    # Encode the text
    encoded = encode_text(test_text)
    
    # Pad the sequence
    padded = pad_sequences([encoded], maxlen=max_len)
    
    # Make prediction
    prediction = model.predict(padded, verbose=0)
    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"
    confidence = prediction[0][0] if prediction[0][0] > 0.5 else 1 - prediction[0][0]
    
    print(f"Text: {test_text}")
    print(f"Prediction: {sentiment} (Confidence: {confidence:.4f})")
    print("-" * 60)


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
Testing BRNN Model on Sample Strings:

------------------------------------------------------------
Testing BRNN Model on Sample Strings:

------------------------------------------------------------
Text: The movie was fantastic! I really loved it.
Prediction: Positive (Confidence: 0.9322)
------------------------------------------------------------
Text: The movie was fantastic! I really loved it.
Prediction: Positive (Confidence: 0.9322)
------------------------------------------------------------
Text: This film is terrible and boring. Waste of time.Text: This film is terrible and boring. Waste of time.
Prediction: Positive (Confidence: 0.9907)
------------------------------------------------------------

Prediction: Positive (Confidence: 0.9907)
------------------------------------------------------------
Text: Amazing performance by the actors. Highly recommended!
Prediction: Positiv